<a href="https://colab.research.google.com/github/BRV12G/Final_year_Project/blob/main/fitness_final_edited_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# ----------------------------------------------------------
# Step 1: # Import required libraries
# ----------------------------------------------------------

import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder ,MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Step 1: Load the Dataset
# Replace 'your_dataset.csv' with the actual path
file_path = '/content/updated_cleaned_diet_split_exercises.csv'
data = pd.read_csv(file_path)
print("Dataset Loaded:")
# print(data.head())
#  Drop the ID column (if it exists)
if 'ID' in data.columns:
    data = data.drop(columns=['ID'])
    print("ID column dropped")

# ----------------------------------------------------------
# Step 2: Dataset Preprocessing
# ----------------------------------------------------------

# ----------------------------------------------------------
#  Handling numeric inputs and outputs
# ----------------------------------------------------------
numeric_cols = ['Age', 'Height', 'Weight', 'Duration_of_Workout']
output_numeric_cols = ['BMI', 'Calories_Burnt', 'Water_Intake(Litres)']

# Impute Missing Values (mean imputation)
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())
data[output_numeric_cols] = data[output_numeric_cols].fillna(data[output_numeric_cols].mean())

# Standardize Numeric Columns
# scaler = StandardScaler()
# Create separate scalers for inputs and outputs
input_scaler = StandardScaler()
output_scaler = StandardScaler()

# Fit and transform the input numeric columns and output numeric columns
data[numeric_cols] = input_scaler.fit_transform(data[numeric_cols])
data[output_numeric_cols] = output_scaler.fit_transform(data[output_numeric_cols])

# joblib.dump(scaler, 'scaler.pkl')
# Save the scalers separately
joblib.dump(input_scaler, 'input_scaler.pkl')
joblib.dump(output_scaler, 'output_scaler.pkl')

print("Numeric Columns after Scaling:")
# print(data[numeric_cols + output_numeric_cols].head())

# ----------------------------------------------------------
# Handling Binary categorical columns using label encoding
# ----------------------------------------------------------

# Binary Encoding for Binary Categorical Columns
binary_cols = ['Gender', 'Hypertension', 'Diabetes']
label_encoders = {}
for col in binary_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le
    joblib.dump(le, f'{col}_encoder.pkl')
print("Binary Encoded Columns:")
# print(data[binary_cols].head())

#-------------------------------------------------------------------------------
 # One hot encoding for categorical columns  fitness goal and fitness type
#------------------------------------------------------------------------------
fitness_goal_encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
fitness_type_encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
# Fit and transform 'Fitness_Goal'
fitness_goal_encoded = fitness_goal_encoder.fit_transform(data[['Fitness_Goal']])
fitness_goal_columns = fitness_goal_encoder.get_feature_names_out(['Fitness_Goal'])
# Create a DataFrame for 'Fitness_Goal' encoded data
fitness_goal_df = pd.DataFrame(fitness_goal_encoded, columns=fitness_goal_columns, index=data.index)
# Fit and transform 'Fitness_Type'
fitness_type_encoded = fitness_type_encoder.fit_transform(data[['Fitness_Type']])
fitness_type_columns = fitness_type_encoder.get_feature_names_out(['Fitness_Type'])
# Create a DataFrame for 'Fitness_Type' encoded data
fitness_type_df = pd.DataFrame(fitness_type_encoded, columns=fitness_type_columns, index=data.index)
# Concatenate the original data with the encoded columns and drop the original columns
data = pd.concat([data, fitness_goal_df, fitness_type_df], axis=1)
data = data.drop(columns=['Fitness_Goal', 'Fitness_Type'])
# Save the OneHotEncoder objects for reuse
joblib.dump(fitness_goal_encoder, 'fitness_goal_encoder.pkl')
joblib.dump(fitness_type_encoder, 'fitness_type_encoder.pkl')
# Save the column names for each encoder
joblib.dump(list(fitness_goal_columns), 'fitness_goal_columns.pkl')
joblib.dump(list(fitness_type_columns), 'fitness_type_columns.pkl')
print("One-Hot Encoded Columns Added:")


#-------------------------------------------------------------------------------
 #label encoding for bmi class since they are ordinal
#------------------------------------------------------------------------------

# Label Encoding for BMI_Class
bmi_class_encoder = LabelEncoder()
data['BMI_Class'] = bmi_class_encoder.fit_transform(data['BMI_Class'])
# Save the LabelEncoder for future use
joblib.dump(bmi_class_encoder, 'bmi_class_encoder.pkl')
print("BMI_Class after Label Encoding:")
# print(data['BMI_Class'].head())

#-------------------------------------------------------------------------------
# one-hot encoding for exercise columns
#------------------------------------------------------------------------------

exercise_cols = ['Exercise1', 'Exercise2', 'Exercise3']
one_hot_encoders = {}  # Dictionary to store OneHotEncoders for each column
# Loop through each exercise column and apply OneHotEncoder
for col in exercise_cols:
    # Initialize OneHotEncoder
    encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
    # Fit and transform the column
    encoded_data = encoder.fit_transform(data[[col]])
    # Add the one-hot encoded columns to the original DataFrame
    encoded_df = pd.DataFrame(
        encoded_data,
        columns=[f"{col}_{cat}" for cat in encoder.categories_[0][1:]]  # Exclude the first category due to 'drop="first"'
    )
    data = pd.concat([data, encoded_df], axis=1)
    # Save the encoder for future use
    one_hot_encoders[col] = encoder
    joblib.dump(encoder, f"{col}_encoder.pkl")
    print(f"One-Hot Encoding completed for {col} and saved as {col}_encoder.pkl.")
# Drop the original exercise columns
data = data.drop(columns=exercise_cols)
print("One-Hot Encoded Exercise Columns Added:")

#-------------------------------------------------------------------------------
# One-Hot Encoding for Equipment Columns
#------------------------------------------------------------------------------

# Define the equipment columns
equipment_cols = ['Equipment1', 'Equipment2']
# Initialize OneHotEncoders for each equipment column
equipment1_encoder = OneHotEncoder(sparse_output=False, drop='first')
equipment2_encoder = OneHotEncoder(sparse_output=False, drop='first')
# Apply OneHotEncoding for 'Equipment1'
equipment1_encoded = equipment1_encoder.fit_transform(data[['Equipment1']])
equipment1_columns = equipment1_encoder.get_feature_names_out(['Equipment1'])
# Add the encoded columns back to the DataFrame
data_equipment1 = pd.DataFrame(equipment1_encoded, columns=equipment1_columns, index=data.index)
data = pd.concat([data, data_equipment1], axis=1)
# Apply OneHotEncoding for 'Equipment2'
equipment2_encoded = equipment2_encoder.fit_transform(data[['Equipment2']])
equipment2_columns = equipment2_encoder.get_feature_names_out(['Equipment2'])
# Add the encoded columns back to the DataFrame
data_equipment2 = pd.DataFrame(equipment2_encoded, columns=equipment2_columns, index=data.index)
data = pd.concat([data, data_equipment2], axis=1)
# Drop the original columns
data = data.drop(columns=equipment_cols)
# Save the encoders and the column names for future use
joblib.dump(equipment1_encoder, 'equipment1_encoder.pkl')
joblib.dump(equipment2_encoder, 'equipment2_encoder.pkl')
joblib.dump(list(equipment1_columns), 'equipment1_columns.pkl')
joblib.dump(list(equipment2_columns), 'equipment2_columns.pkl')

print("One-Hot Encoded Equipment Columns Added:")


# ----------------------------------------------------------
# multi-hot encoding  for Text Columns
# ----------------------------------------------------------

# Step 1: Initialize MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Step 2: Process each text column
mlb_dict = {}  # Dictionary to store MultiLabelBinarizer for each text column
text_columns = ['d_vegetables', 'd_juice', 'd_proteinintake']
for col in text_columns:
    print(f"Processing {col}...")

    # Split by commas and strip spaces
    split_data = data[col].fillna("").str.split(", ").apply(lambda x: [item.strip() for item in x])

    # MultiHot encoding using MultiLabelBinarizer
    multi_hot_encoded = mlb.fit_transform(split_data)

    # Save the fitted MultiLabelBinarizer for reuse
    mlb_dict[col] = mlb
    joblib.dump(mlb, f'{col}_mlb.pkl')  # Save each MultiLabelBinarizer separately

    # Create a DataFrame with multi-hot encoded features
    multi_hot_df = pd.DataFrame(multi_hot_encoded, columns=[f"{col}_{item}" for item in mlb.classes_])

    # Merge the multi-hot encoded columns with the main DataFrame
    data = pd.concat([data, multi_hot_df], axis=1)

# Step 3: Drop the original text columns
data = data.drop(columns=text_columns)

# Display the processed DataFrame
print("Text Columns Processed with Multi-Hot Encoding:")
# print(data.head())


#-----------------------------------------------------------------------------
#  Save the Preprocessed Dataset
#-----------------------------------------------------------------------------
preprocessed_file_path = '/content/preprocessed_dataset.csv'
data.to_csv(preprocessed_file_path, index=False)

print(f"Preprocessed dataset saved at: {preprocessed_file_path}")


#--------------------------------------------------------------------------------
#--------------------------------------------------------------------------------
#--------------------------------------------------------------------------------

# ----------------------------------------------------------
# Step 3: Splitting Data into Inputs (X) and Outputs (y)
# ----------------------------------------------------------

# Define Input Columns (features)
input_features = ['Gender', 'Age', 'Height', 'Weight', 'Hypertension', 'Diabetes', 'Duration_of_Workout'] + \
                 list(data.filter(like='Fitness_Goal_')) + \
                 list(data.filter(like='Fitness_Type_'))



output_features = ['BMI', 'BMI_Class'] + \
                 list(data.filter(like='Exercise1_')) + \
                 list(data.filter(like='Exercise2_')) + \
                 list(data.filter(like='Exercise3_')) + ['Calories_Burnt', 'Water_Intake(Litres)'] + \
                 list(data.filter(like='Equipment1_')) + \
                 list(data.filter(like='Equipment2_')) + \
                 list(data.filter(like='d_'))

X = data[input_features]
y = data[output_features]

print("Input Features Shape:", X.shape)
print("Output Features Shape:", y.shape)



# ----------------------------------------------------------
# Step 4: Splitting Data into Train, Validation, and Test Sets
# ----------------------------------------------------------

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Train Shape:", X_train.shape, y_train.shape)
print("Validation Shape:", X_val.shape, y_val.shape)
print("Test Shape:", X_test.shape, y_test.shape)



#-------------------------------------------------------------------------------
# ----------------------------------------------------------
# Step 5: Training , building and compiling the model
# ----------------------------------------------------------
#-------------------------------------------------------------------------------

# Import necessary libraries for deep learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import load_model


# Step 1: Build the Deep Learning Model
model = Sequential()

# Input Layer
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))

# Hidden Layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))  # Dropout layer for regularization
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))


# Output Layer
model.add(Dense(y_train.shape[1], activation='linear'))  # Linear activation for regression tasks

# Step 2: Compile the Model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

# Step 3: Train the Model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))


#-------------------------------------------------------------------------------
# # Step 6: Evaluate the Model on training , testing and validation sets
#-------------------------------------------------------------------------------
#train data evaluation
mean_train_loss = sum(history.history['loss']) / len(history.history['loss'])
mean_train_mae = sum(history.history['mae']) / len(history.history['mae'])
print(f"Mean Train Loss: {mean_train_loss:.4f}")
print(f"Mean Train MAE: {mean_train_mae:.4f}")

#validation data evaluation
mean_val_loss = sum(history.history['val_loss']) / len(history.history['val_loss'])
mean_val_mae = sum(history.history['val_mae']) / len(history.history['val_mae'])
print(f"Mean Validation Loss: {mean_val_loss:.4f}")
print(f"Mean Validation MAE: {mean_val_mae:.4f}")

#test data evaluation
loss, mae = model.evaluate(X_test, y_test)
print(f"Model Evaluation - Loss: {loss:.4f}, Mean Absolute Error: {mae:.4f}")


model.save('trained_deep_nn_model.h5')

print("Model has been saved as 'trained_deep_nn_model.h5'")

#----------------------------------------------------------------------------
# step 7: testing the model on user inputs
#----------------------------------------------------------------------------

import pandas as pd
import joblib
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import load_model
import numpy as np

# Load the encoders and scalers saved during training
gender_encoder = joblib.load('/content/Gender_encoder.pkl')
hypertension_encoder = joblib.load('/content/Hypertension_encoder.pkl')
diabetes_encoder = joblib.load('/content/Diabetes_encoder.pkl')

fitness_goal_encoder = joblib.load('/content/fitness_goal_encoder.pkl')
fitness_type_encoder = joblib.load('/content/fitness_type_encoder.pkl')

input_scaler = joblib.load('/content/input_scaler.pkl')
output_scaler = joblib.load('/content/output_scaler.pkl')
bmi_class_encoder = joblib.load('/content/bmi_class_encoder.pkl')

exercise_encoders = {col: joblib.load(f"/content/{col}_encoder.pkl") for col in ['Exercise1', 'Exercise2', 'Exercise3']}
equipment_encoders = {
    'Equipment1': joblib.load('equipment1_encoder.pkl'),
    'Equipment2': joblib.load('equipment2_encoder.pkl')
}

# Load the pre-trained MultiLabelBinarizer objects
mlb_vegetables = joblib.load('/content/d_vegetables_mlb.pkl')
mlb_juice = joblib.load('/content/d_juice_mlb.pkl')
mlb_proteinintake = joblib.load('/content/d_proteinintake_mlb.pkl')

mlb_dict = {col: joblib.load(f'/content/{col}_mlb.pkl') for col in ['d_vegetables', 'd_juice', 'd_proteinintake']}

# Load the trained deep neural network model
model = load_model('/content/trained_deep_nn_model.h5')

# Example user input
user_input = {
    'Gender': ['Female'],
    'Age': [18],
    'Height': [168],
    'Weight': [47.5],
    'Hypertension': ['Yes'],
    'Diabetes': ['No'],
    'Duration_of_Workout': [35],
    'Fitness_Goal': ['Weight Gain'],
    'Fitness_Type': ['Cardio'],
}

# Step 1: Preprocess the input data
user_input_df = pd.DataFrame(user_input)

# Apply label encoding
user_input_df['Gender'] = gender_encoder.transform(user_input_df['Gender'])
user_input_df['Hypertension'] = hypertension_encoder.transform(user_input_df['Hypertension'])
user_input_df['Diabetes'] = diabetes_encoder.transform(user_input_df['Diabetes'])

# Apply one-hot encoding
fitness_goal_encoded = fitness_goal_encoder.transform(user_input_df[['Fitness_Goal']])
fitness_type_encoded = fitness_type_encoder.transform(user_input_df[['Fitness_Type']])

fitness_goal_df = pd.DataFrame(fitness_goal_encoded, columns=fitness_goal_encoder.get_feature_names_out())
fitness_type_df = pd.DataFrame(fitness_type_encoded, columns=fitness_type_encoder.get_feature_names_out())

# Merge encoded columns and drop the originals
user_input_df = pd.concat([user_input_df, fitness_goal_df, fitness_type_df], axis=1)
user_input_df = user_input_df.drop(columns=['Fitness_Goal', 'Fitness_Type'])

# Scale numeric columns
numeric_columns = ['Age', 'Height', 'Weight', 'Duration_of_Workout']
user_input_df[numeric_columns] = input_scaler.transform(user_input_df[numeric_columns])

# Step 2: Make predictions
predictions = model.predict(user_input_df)

print(predictions.shape)
# print(predictions[:5, :])
print(predictions)

import pandas as pd
import numpy as np

# Define the columns corresponding to the predictions
input_columns = ['Gender', 'Age', 'Height', 'Weight', 'Hypertension', 'Diabetes', 'Duration_of_Workout', 'Fitness_Goal', 'Fitness_Type']
output_columns = ['BMI', 'BMI_Class', 'Exercise1', 'Exercise2', 'Exercise3', 'Calories_Burnt', 'Water_Intake(Litres)', 'Equipment1', 'Equipment2',
                  'd_vegetables', 'd_juice', 'd_proteinintake']

# Assuming predictions is the array of predictions
predictions = predictions[0]  # Remove the batch dimension, shape becomes (103,)

# Separate predictions into the input and output parts
input_predictions = predictions[:len(input_columns)]  # First part corresponds to inputs
output_predictions = predictions[len(input_columns):]  # Remaining part corresponds to outputs


# Assuming positions in output_scaler: BMI = 0, Calories_Burnt = 1, Water_Intake(Litres) = 2

output_scaler_columns = ['BMI', 'Calories_Burnt', 'Water_Intake(Litres)']

# Extract the specific positions for each feature
bmi_value = output_scaler.inverse_transform([[output_predictions[0], 0, 0]])[0][0]  # Only BMI value
calories_burnt = output_scaler.inverse_transform([[0, output_predictions[4], 0]])[0][1]  # Only Calories_Burnt value
water_intake = output_scaler.inverse_transform([[0, 0, output_predictions[5]]])[0][2]  # Only Water_Intake(Litres)
bmi_class_value = bmi_class_encoder.inverse_transform([np.argmax(output_predictions[1:4])])[0]

# Print the decoded values
print("Decoded BMI Value:", bmi_value)
print("Decoded Calories Burnt:", calories_burnt)
print("Decoded Water Intake (Litres):", water_intake)
print("Decoded BMI Class:", bmi_class_value)

# Process One-Hot Encoded Outputs (like Exercise1, Exercise2, Exercise3, Equipment1, Equipment2)
exercise_predictions = output_predictions[:5 * len(['Exercise1', 'Exercise2', 'Exercise3'])]  # Assuming 3 classes for each exercise
equipment_predictions = output_predictions[-len(['Equipment1', 'Equipment2']) * 5:]  # Assuming Equipment1, Equipment2 are one-hot encoded

# Inverse transform categorical predictions
decoded_exercises = []
for i, col in enumerate(['Exercise1', 'Exercise2', 'Exercise3']):
    start_idx = i * 5
    end_idx = (i + 1) * 5
    exercise_probs = exercise_predictions[start_idx:end_idx]
    # Get the predicted class index (index with max probability)
    predicted_exercise_index = np.argmax(exercise_probs)
    # Create a one-hot vector with the max index set to 1
    one_hot_vector = [0] * 5  # Assuming 6 classes for exercises
    one_hot_vector[predicted_exercise_index] = 1
    # Inverse transform to get the actual exercise class
    predicted_exercise = exercise_encoders[col].inverse_transform([one_hot_vector])
    decoded_exercises.append(predicted_exercise[0][0])
    print("Decoded Exercises Predictions:")
    print(decoded_exercises)


decoded_equipment = []
for i, col in enumerate(['Equipment1', 'Equipment2']):
    start_idx = i * 5
    end_idx = (i + 1) * 5
    equipment_probs = equipment_predictions[start_idx:end_idx]
    # Get the predicted class index (index with max probability)
    predicted_equipment_index = np.argmax(equipment_probs)
    # Create a one-hot vector with the max index set to 1
    one_hot_vector = [0] * 5  # Assuming 6 classes for equipment
    one_hot_vector[predicted_equipment_index] = 1
    # Inverse transform to get the actual equipment class
    predicted_equipment = equipment_encoders[col].inverse_transform([one_hot_vector])
    decoded_equipment.append(predicted_equipment[0][0])
    print("Decoded Equipment Predictions:")
    print(decoded_equipment)


# Extract the multi-hot encoded values for each category
multi_hot_vegetables = output_predictions[6:6 + 29]  # 29 unique vegetables
multi_hot_juice = output_predictions[6 + 29:6 + 29 + 15]  # 15 unique juice items
multi_hot_protein = output_predictions[6 + 29 + 15:6 + 29 + 15 + 30]  # 30 unique protein intake items

# Threshold the multi-hot encoded outputs (e.g., using 0.5 as the threshold)
threshold_vegetables = (multi_hot_vegetables > 0.5).astype(int)
threshold_juice = (multi_hot_juice > 0.5).astype(int)
threshold_protein = (multi_hot_protein > 0.5).astype(int)

# Inverse transform the thresholded multi-hot values
decoded_vegetables = mlb_vegetables.inverse_transform(np.array([threshold_vegetables]))[0]
decoded_juice = mlb_juice.inverse_transform(np.array([threshold_juice]))[0]
decoded_proteinintake = mlb_proteinintake.inverse_transform(np.array([threshold_protein]))[0]


# Now you have decoded results as lists of categories
print("Decoded Vegetables:", decoded_vegetables)
print("Decoded Juice:", decoded_juice)
print("Decoded Protein Intake:", decoded_proteinintake)

print ("predictions completed")



Dataset Loaded:
ID column dropped
Numeric Columns after Scaling:
Binary Encoded Columns:
One-Hot Encoded Columns Added:
BMI_Class after Label Encoding:
One-Hot Encoding completed for Exercise1 and saved as Exercise1_encoder.pkl.
One-Hot Encoding completed for Exercise2 and saved as Exercise2_encoder.pkl.
One-Hot Encoding completed for Exercise3 and saved as Exercise3_encoder.pkl.
One-Hot Encoded Exercise Columns Added:
One-Hot Encoded Equipment Columns Added:
Processing d_vegetables...
Processing d_juice...
Processing d_proteinintake...
Text Columns Processed with Multi-Hot Encoding:
Preprocessed dataset saved at: /content/preprocessed_dataset.csv
Input Features Shape: (14589, 18)
Output Features Shape: (14589, 103)
Train Shape: (11671, 18) (11671, 103)
Validation Shape: (1459, 18) (1459, 103)
Test Shape: (1459, 18) (1459, 103)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
365/365 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.1588 - mae: 0.2469 - val_loss: 0.0903 - val_mae: 0.1754
Epoch 2/50
365/365 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0959 - mae: 0.1875 - val_loss: 0.0794 - val_mae: 0.1563
Epoch 3/50
365/365 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0858 - mae: 0.1721 - val_loss: 0.0733 - val_mae: 0.1452
Epoch 4/50
365/365 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0794 - mae: 0.1611 - val_loss: 0.0692 - val_mae: 0.1359
Epoch 5/50
365/365 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0767 - mae: 0.1559 - val_loss: 0.0675 - val_mae: 0.1321
Epoch 6/50
365/365 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0739 - mae: 0.1520 - val_loss: 0.0665 - val_mae: 0.1303
Epoch 7/50
365/365 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0734 - mae: 0.1508 - val_loss: 0.0654 - val_mae: 0.1282
Epoch 8/50
365/365 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0713 - mae: 0.1480 - val_loss: 0.0656 - val_mae: 0.1318
Epoch 9/50
365/365 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - lo

Model Evaluation - Loss: 0.0604, Mean Absolute Error: 0.1182
Model has been saved as 'trained_deep_nn_model.h5'


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
(1, 103)
[[-1.19307387e+00  2.97843981e+00  2.20284760e-01  1.56005785e-01
   1.49590030e-01  1.77391171e-01  1.39731795e-01  1.55541852e-01
   1.49590552e-01  1.39740050e-01  1.56013221e-01  2.20295817e-01
   1.49569586e-01  1.55546799e-01  1.56009391e-01  1.39727056e-01
   2.20293984e-01 -8.39573681e-01  6.63359649e-03  1.90656766e-01
   1.90625414e-01  1.43911585e-01  1.41592696e-01  1.64635003e-01
   1.43926054e-01  1.64645135e-01  1.90647945e-01  1.90654054e-01
   1.41576469e-01  3.13526019e-02 -1.49679184e-02 -1.56320632e-04
   2.56778635e-02  1.38258487e-02  9.73050594e-01 -5.43811917e-03
  -1.01372004e-02 -2.25950778e-03 -7.87167996e-03 -1.01464987e-02
  -1.49681717e-02  2.60355622e-02 -1.49676651e-02  1.94626935e-02
   9.41257358e-01 -1.58503652e-04  1.79561004e-02  9.41271245e-01
  -7.87221640e-03  9.41271245e-01  9.35108960e-01 -7.87261128e-03
  -7.87229836e-03 -7.87270814e-03 -2.43968517e-02  4.25855815e-02
   9.41259682e-01  1.81198269